In [3]:
import pandas as pd
import gzip
import json
import re
import datetime
from math import floor
import numpy as np
from random import randint
from collections import Counter

In [4]:
class Coinmarketcap(object):
    @staticmethod
    def load_data(source_datapath):
        by_date_data = {}
        all_data = pd.read_csv(source_datapath, sep='\t', header=0, encoding="utf-8", dtype={'Market Cap': np.float64, 'Total Market Cap': np.float64})
        all_data["Market Cap Share"] = all_data["Market Cap"]/all_data["Total Market Cap"]
        for index, value in all_data['Date'].items():
            date = value
            symbol = all_data['Symbol'][index]
            price = all_data['Price'][index]
            marketcap = all_data['Market Cap'][index]
            position = all_data['Pos'][index]
            if by_date_data.get(date) is None:
                by_date_data[date] = {'symbols': {}, 'positions': []}
            by_date_data[date]['symbols'][symbol] = {
                "price": price,
                "marketcap": marketcap
            }
            by_date_data[date]['positions'].append([symbol, position])
        for date in by_date_data.keys():
            by_date_data[date]['positions'] = [el[0] for el in sorted(by_date_data[date]['positions'], key=lambda x: x[1])]
        return all_data, by_date_data

    def __init__(self, source_datapath):
        self.data, self.by_date_data = self.load_data(source_datapath)
    
    def get_coin_price(self, date, coin_symbol):
        try:
            return float(self.by_date_data[date]['symbols'][coin_symbol]['price'])
        except:
            #print("ERROR: price not found for coin %s (%s)" % (coin_symbol, date))
            return 0

    def coins_to_usd(self, date, coins):
        amount_usd = 0
        for coin_symbol, amount in coins.items():
            coin_usd = self.get_coin_price(date, symbol)
            amount_usd += amount * coin_usd
        return amount_usd
    
    def buy_coin(self, date, symbol, amount_usd):
        try:
            return amount_usd/self.get_coin_price(date, symbol)
        except:
            print(date, symbol, amount_usd)
            raise Exception("")
    
    def buy_coins(self, date, symbols, amount_usd, strategy="even"):
        symbols_length = len(symbols)
        coins = {}
        if strategy == "market_cap":
            coins_market_cap = {}
            total_market_cap = 0
            for coin_symbol in symbols:
                coins_market_cap[coin_symbol] = self.get_coin_market_cap(date, coin_symbol)
                total_market_cap += coins_market_cap[coin_symbol]
        for coin_symbol in symbols:
            coin_amount_usd = 0
            if strategy == "even":
                coin_amount_usd = (amount_usd/symbols_length)
            elif strategy == "market_cap":
                coin_amount_usd = (amount_usd*(coins_market_cap[coin_symbol]/total_market_cap))
            coins[coin_symbol] = self.buy_coin(date, coin_symbol, coin_amount_usd)
        return coins
    
    def sell_coins(self, date, coins):
        amount_usd = 0
        for coin_symbol, coin_amount in coins.items():
            coin_price = self.get_coin_price(date, coin_symbol)
            amount_usd += coin_price*coin_amount
        return amount_usd
    
    def get_coin_market_cap(self, date, coin_symbol):
        return float(self.by_date_data[date]['symbols'][coin_symbol]['marketcap'])
            
    def get_positions_range(self, date, first_position, last_position):
        return self.by_date_data[date]['positions'][first_position-1:last_position]
    
    def dates(self):
        return sorted(self.by_date_data.keys())
    
    def print_returns(self, initial_amount, initial_date, final_amount, final_date):
        print("%s: %s USD" % (initial_date, round(initial_amount, 2)))
        print("%s: %s USD" % (end_date, round(final_amount, 2)))
        print("----------")
        print("%sX" % round(final_amount/initial_amount, 1))
        

In [9]:
class InvestStrategies(object):
    def __init__(self, market, options):
        self.market = market
        self.options = options
        self.dates = self.market.dates()

    def buy(self, amount, date):
        raise Exception("Implementation needed")
        

    def get_random_start_end_date(self):
        min_week_duration = self.options.get('min_week_duration') or 1
        max_week_duration = self.options.get('max_week_duration') or 52
        min_start_date = self.options.get('min_start_date', self.dates[0])
        max_end_date = self.options.get('max_end_date', self.dates[-1])
        
        min_week_index = min([index for index, date in enumerate(self.dates) if min_start_date <= date])
        max_week_index = max([index for index, date in enumerate(self.dates) if max_end_date >= date])
        week_duration = randint(min_week_duration, max_week_duration)
        
        random_start_index = randint(min_week_index, max_week_index - week_duration)
        random_end_index = random_start_index + week_duration
        duration = random_end_index - random_start_index
        return duration, random_start_index, random_end_index

    def buy_top_x(self, date, amount, top_x_first_pos, top_x_last_pos, strategy, exclude=None):
        if exclude is None:
            exclude = []
        symbols_to_buy = self.market.get_positions_range(date, top_x_first_pos, top_x_last_pos)
        symbols_to_buy = [symbol for symbol in symbols_to_buy if symbol not in exclude]
        return self.market.buy_coins(date, list(symbols_to_buy), amount, strategy=strategy)
    
    def add_coins_list(self, coins_group):
        coins_keys = set([
            coin_key
            for coins_data in coins_group
            for coin_key in coins_data.keys()
            
        ])
        coins = {}
        for coins_data in coins_group:
            for coin_key in coins_keys:
                if coins_data.get(coin_key):
                    if coins.get(coin_key) is None:
                        coins[coin_key] = 0
                    coins[coin_key] += coins_data[coin_key]
        return coins

    def run_strategy(self, initial_amount, balance_x_weeks=0):
        iterations = self.options.get('iterations', 10)
        results = {
            "min_date": "",
            "max_date": "",
            "min_duration": "",
            "max_duration": "",
            "values": [],
            "bought_coins": [],
            "raw_values": []
        }
        bought_coins = []
        for i in range(0, iterations):
            weeks_duration, start_date_index, end_date_index = self.get_random_start_end_date()
            if not results['max_duration'] or results['max_duration'] < weeks_duration:
                results['max_duration'] = weeks_duration
            if not results['min_duration'] or results['min_duration'] > weeks_duration:
                results['min_duration'] = weeks_duration
            
            if not results['min_date'] or results['min_date'] > self.dates[start_date_index]:
                results['min_date'] = self.dates[start_date_index]
            if not results['max_date'] or results['max_date'] < self.dates[end_date_index]:
                results['max_date'] = self.dates[end_date_index]
            
            amount = initial_amount
            if balance_x_weeks > 0:
                first_date = True
                coins = {}
                for date in self.dates[start_date_index:end_date_index:balance_x_weeks]:
                    if not first_date:
                        amount = market.sell_coins(date, coins)
                    else:
                        first_date = False
                    coins = self.buy(amount, date)
                    bought_coins += coins.keys()
                amount = market.sell_coins(date, coins)
            else:
                coins = self.buy(amount, self.dates[start_date_index])
                bought_coins += coins.keys()
                amount = market.sell_coins(self.dates[end_date_index], coins)
            
            results['raw_values'].append(
                [
                    self.dates[start_date_index],
                    self.dates[end_date_index],
                    weeks_duration,
                    round(1.0*amount/initial_amount, 2),
                    round(100.0*(amount/initial_amount - 1)/weeks_duration, 3)
                ]
            )
                    

        results['iterations'] = iterations
        results['values'] = sorted([value for _, _, _, _, value in results['raw_values']])
        results['mean'] = round(np.mean(results['values']), 2)
        results['median'] = round(np.median(results['values']), 2)
        results['min_rate'] = round(np.min(results['values']), 2)
        results['max_rate'] = round(np.max(results['values']), 2)
        results['bought_coins'] = Counter(bought_coins)
        results['total_bought_coins'] = len(Counter(bought_coins))
        results['raw_values'] = sorted(results['raw_values'], key=lambda x: x[4])
        results['median_values'] = [
            values for values in results['raw_values'] if values[4] < results['median']
        ][-1]
        return results
    
    def print_results(self, results):
        print("\n\n%s:\n" % self.name())
        for key in [
            'min_rate', 'max_rate', 'median_values'
            #, 'total_bought_coins', 'bought_coins'
        ]:
            #TODO: refactor this
            if key == "min_rate":
                print(
                    "Min return(%s to %s) %s weeks: %sx. %sx yearly" % (
                        results['raw_values'][0][0], results['raw_values'][0][1],
                        results['raw_values'][0][2], results['raw_values'][0][3],
                        round(52*results['raw_values'][0][3]/results['raw_values'][0][2], 2)
                    )
            )
            elif key == "max_rate":
                print(
                    "Max return(%s to %s) %s weeks: %sx. %sx yearly" % (
                        results['raw_values'][-1][0], results['raw_values'][-1][1],
                        results['raw_values'][-1][2], results['raw_values'][-1][3],
                        round(52*results['raw_values'][-1][3]/results['raw_values'][-1][2], 2)
                    )
            )
            elif key == "median_values":
                print(
                    "Mean return(%s to %s) %s weeks: %sx. %sx yearly" % (
                        results[key][0], results[key][1],
                        results[key][2], results[key][3],
                        round(52*results[key][3]/results[key][2], 2)
                    )
            )
            elif key == "bought_coins":
                print("%s: %s" % (key, str(list(results[key].keys()))))
            else:
                print("%s: %s" % (key, results[key]))
        
        
        

# Initialize

In [6]:
market = Coinmarketcap("data/coinmarketcap_formatted.tsv")

In [14]:
strategy_options = {
    'min_week_duration': 40,
    'max_week_duration': 60,
    'min_start_date': '2015-09-01',
    'iterations': 1000
}

We will implement several strategies, and run them over several dates, from not before 2015-09-01 and not after 2017-04-30. We will run 1000 iterations with a minimun legth of 16 weeks (from first investment and last)

In [16]:
class OnlyBitcoin(InvestStrategies):
    def buy(self, amount, date):
        return self.buy_top_x(date, amount, top_x_first_pos=1, top_x_last_pos=1, strategy="even")

    def name(self):
        return "OnlyBitcoin"

    
class Top5Evenly(InvestStrategies):
    def buy(self, amount, date):
        return self.buy_top_x(date, amount, top_x_first_pos=1, top_x_last_pos=5, strategy="even")

    def name(self):
        return "Top 5 in same proportion"

class Top5MarketCap(InvestStrategies):
    def buy(self, amount, date):
        return self.buy_top_x(date, amount, top_x_first_pos=1, top_x_last_pos=5, strategy="market_cap")

    def name(self):
        return "Top 5 by market cap"

class Top1_50_Top2_5_50(InvestStrategies):
    def buy(self, amount, date):
        top1_coins = self.buy_top_x(date, 1.0*amount/2, top_x_first_pos=1, top_x_last_pos=1, strategy="even")
        top2_5_coins = self.buy_top_x(date, 1.0*amount/2, top_x_first_pos=2, top_x_last_pos=5, strategy="even")
        coins = {**top1_coins, **top2_5_coins}
        return coins

    def name(self):
        return "Top1 50%, Top2-5 50% by marketcap"

class TopX(InvestStrategies):
    def buy(self, amount, date):
        coins_group = []
        coins_group.append(
            self.buy_top_x(date, 0.4*amount, top_x_first_pos=1, top_x_last_pos=1, strategy="even")
        )
        coins_group.append(
            self.buy_top_x(date, 0.3*amount, top_x_first_pos=2, top_x_last_pos=2, strategy="even")
        )
        coins_group.append(
            self.buy_top_x(date, 0.2*amount, top_x_first_pos=3, top_x_last_pos=5, strategy="even")
        )
        coins_group.append(
            self.buy_top_x(date, 0.1*amount, top_x_first_pos=3, top_x_last_pos=10, strategy="even")
        )
        coins = self.add_coins_list(coins_group)
        return coins

    def name(self):
        return "Top1 40%, TOP2 30%, Top3-5 20% evenly, Top3-10 20% evenly"

    
class TopY(InvestStrategies):
    def buy(self, amount, date):
        coins_group = []
        coins_group.append(
            self.buy_top_x(date, 0.35*amount, top_x_first_pos=1, top_x_last_pos=1, strategy="even")
        )
        coins_group.append(
            self.buy_top_x(date, 0.25*amount, top_x_first_pos=2, top_x_last_pos=2, strategy="even")
        )
        coins_group.append(
            self.buy_top_x(date, 0.4*amount, top_x_first_pos=3, top_x_last_pos=10, strategy="market_cap")
        )
        coins = self.add_coins_list(coins_group)
        return coins

    def name(self):
        return "Top1 35%, TOP2 25%, Top3-10 40% market_cap"
    
    
class TopZ(InvestStrategies):
    def buy(self, amount, date):
        coins_group = []
        coins_group.append(
            {"BTC": self.market.buy_coin(date, "BTC", amount*0.35)}
        )
        coins_group.append(
            {"ETH": self.market.buy_coin(date, "ETH", amount*0.25)}
        )
        coins_group.append(
            {"DASH": self.market.buy_coin(date, "DASH", amount*0.05)}
        )
        coins_group.append(
            {"XRP": self.market.buy_coin(date, "XRP", amount*0.05)}
        )
        coins_group.append(
            self.buy_top_x(
                date, 0.3*amount, top_x_first_pos=3, top_x_last_pos=10,
                strategy="even", exclude=["BTC", "ETH", "DASH", "XRP"]
            )
        )
        coins = self.add_coins_list(coins_group)
        return coins

    def name(self):
        return "BTC 35%, ETH 25%, DASH 5%, RIPPLE 5%, Top3-10 20% evenly (Without already bought). Could have a cherry picking problem"


    
print("Test parameters", json.dumps(strategy_options, indent=2))

for class_ in [OnlyBitcoin, Top5Evenly, Top5MarketCap, Top1_50_Top2_5_50, TopX, TopY, TopZ]:
    inv = class_(market, strategy_options)
    results = inv.run_strategy(1000, balance_x_weeks=1)
    inv.print_results(results)


Test parameters {
  "min_week_duration": 40,
  "max_week_duration": 60,
  "min_start_date": "2015-09-01",
  "iterations": 1000
}


OnlyBitcoin:

Min return(2016-06-19 to 2017-04-02) 40 weeks: 1.29x. 1.68x yearly
Max return(2015-09-20 to 2016-06-26) 40 weeks: 3.25x. 4.22x yearly
Mean return(2016-05-01 to 2017-03-26) 46 weeks: 2.24x. 2.53x yearly


Top 5 in same proportion:

Min return(2016-03-13 to 2016-12-25) 40 weeks: 0.78x. 1.01x yearly
Max return(2015-10-25 to 2016-07-31) 40 weeks: 3.5x. 4.55x yearly
Mean return(2016-01-17 to 2017-02-12) 55 weeks: 2.12x. 2.0x yearly


Top 5 by market cap:

Min return(2015-12-20 to 2016-10-16) 42 weeks: 1.4x. 1.73x yearly
Max return(2015-09-13 to 2016-06-26) 41 weeks: 3.14x. 3.98x yearly
Mean return(2016-03-20 to 2017-02-12) 46 weeks: 2.21x. 2.5x yearly


Top1 50%, Top2-5 50% by marketcap:

Min return(2016-03-13 to 2016-12-25) 40 weeks: 1.12x. 1.46x yearly
Max return(2016-04-10 to 2017-04-30) 54 weeks: 3.88x. 3.74x yearly
Mean return(2015-11-15 to 20

In [27]:
""

''